# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from tqdm import tqdm
from scipy.spatial.distance import pdist, squareform

In [3]:
#import data
data = pd.read_csv('../data/customer_product_sales.csv')

#and take a look
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
step1 = data.groupby(by=['CustomerID','ProductName']).sum().Quantity.reset_index()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
step2 = pd.pivot_table(data=step1, values='Quantity', index='ProductName', columns='CustomerID', aggfunc='sum', fill_value=0)
step2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [42]:
distances = squareform(pdist(step2.T,metric='euclidean'))

#step3 será uma dataframe das distâncias nas N dimensoes de cada CustomerID para cada CustomerID
step3 = pd.DataFrame(data=distances, columns=step2.columns, index=step2.columns)

In [ ]:
"""
tem dois jeitos de ver similaridade:
1 - Se usamos distancia = distancia, quer dizer que quanto menor esse valor, menos distante os pontos estão
2 - Se usamos distancia = 1/(1+distancia), quer dizer que quanto menor esse valor, maior distante os pontos estão
"""

In [43]:
#quanto maior o valor mais próximo...
step3 = pd.DataFrame(data=1/(1+distances), columns=step2.columns, index=step2.columns)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [56]:
# EXAMPLE FOR FIRST LINE
step3.iloc[0,:].sort_values(ascending=False)[0:6]

CustomerID
33      1.000000
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

In [74]:
#defining a function to get the 5 most similar customersID with a choosen ID
def top_5(step3, customer):
    return step3.loc[:,customer].sort_values(ascending=False)[1:6].index

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [75]:
#we choose:
first_5 = top_5(step3,33)

#filter DF with mask
step5 = step1.loc[step1.loc[:,'CustomerID'].isin(first_5)]
step5

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [90]:
step6= step5.groupby(by='ProductName').sum().loc[:,['Quantity']].sort_values(by='Quantity',ascending=False)
step6

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [103]:
step7 = pd.merge(step6,step2.loc[:,33],on='ProductName')

# products_for_client_33:
list(step7.loc[step7.loc[:,33]==0].index[0:5])

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [107]:
def get_sugestions(custID):
    step5 = step1.loc[step1.loc[:,'CustomerID'].isin(top_5(step3,custID))]
    step6 = step5.groupby(by='ProductName').sum()['Quantity'].sort_values(ascending=False)
    step7 = pd.merge(step6,step2.loc[:,custID],on='ProductName')
    
    return list(step7.loc[step7.loc[:,custID]==0].index[0:5])

In [111]:
recommendations = {}
IDs = data.CustomerID.unique()

for idnum in tqdm(IDs):
    recommendations[idnum] = get_sugestions(idnum)

100%|██████████| 1000/1000 [00:43<00:00, 22.79it/s]


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [123]:
step9 = pd.DataFrame(recommendations).T.sort_index().reset_index()
step9.columns = ['CustomerID','Suggestion 1','Suggestion 2','Suggestion 3','Suggestion 4','Suggestion 5']
step9.head()

,CustomerID,Suggestion 1,Suggestion 2,Suggestion 3,Suggestion 4,Suggestion 5
0,33,Butter - Unsalted,Soup - Campbells Bean Medley,Wine - Ej Gallo Sierra Valley,Wine - Blue Nun Qualitatswein,Scallops 60/80 Iqf
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,"Fish - Scallops, Cold Smoked",Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,Fondant - Icing
3,356,"Veal - Inside, Choice",Sardines,Lamb - Ground,Butter - Unsalted,Beef - Top Sirloin
4,412,"Sauce - Gravy, Au Jus, Mix",Cheese - Cambozola,Olive - Spread Tapenade,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, cosine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
"""
Eu nunca usaria euclideana num problema desses pois ela ignoraria o efeito de dimensões com valores menores em relação
a dimensões com baixos valores. Nesse lab a melhor seria manhattan (cityblock).

André Dedão
"""

In [ ]:
distances = squareform(pdist(step2.T,metric='cityblock'))
step3 = pd.DataFrame(data=distances, columns=step2.columns, index=step2.columns)
step3.head()

In [ ]:
recommendations = {}
IDs = data.CustomerID.unique()[0:5]

for idnum in tqdm(IDs):
    recommendations[idnum] = get_sugestions(idnum)

In [146]:
step9 = pd.DataFrame(recommendations).T.sort_index().reset_index()
step9.columns = ['CustomerID','Suggestion 1','Suggestion 2','Suggestion 3','Suggestion 4','Suggestion 5']
step9.head()

,CustomerID,Suggestion 1,Suggestion 2,Suggestion 3,Suggestion 4,Suggestion 5
0,23548,Chicken - Soup Base,Juice - Orange,Olives - Kalamata,"Tart Shells - Sweet, 4",Tahini Paste
1,40094,"Turnip - White, Organic",Scallops - 10/20,"Wine - White, Schroder And Schyl","Yogurt - Blueberry, 175 Gr",Cheese - Camembert
2,61288,Bagel - Plain,Cream Of Tartar,"Garlic - Primerba, Paste",Juice - Orange,Olive - Spread Tapenade
3,77352,Cream Of Tartar,Cheese - Cheddarsliced,"Garlic - Primerba, Paste","Pasta - Penne, Rigate, Dry",V8 - Berry Blend
4,78981,Onion Powder,"Rosemary - Primerba, Paste",Spinach - Baby,Ocean Spray - Kiwi Strawberry,Yogurt - French Vanilla
